### FORWARD CHAINING

In [ ]:
def read_rules():
    rules = []
    with open("rules.txt", 'r') as file:
        for line in file:
            line = line.strip()
            if 'IF' in line and 'THEN' in line:
                _, condition_and_conclusion = line.split('IF')
                condition_part, conclusion = condition_and_conclusion.split('THEN')
                conclusion = conclusion.strip()
                or_part = condition_part.split('OR')
                for or_cond in or_part:
                    conditions = [c.strip() for c in or_cond.split('AND')]
                    rules.append({'conditions': conditions, 'conclusion': conclusion})
    return rules


def read_facts():
    with open("facts.txt", 'r') as file:
        return [line.strip() for line in file]


def forward(rules, facts, goal=None):
    known_facts = set(facts)
    new_fact = True
    cycle = 1
    while new_fact:
        print(f"\nCycle {cycle}: {known_facts}")
        new_fact = False

        for rule in rules:
            for cond in rule['conditions']:
                matched = False

                if any(op in cond for op in ['=', '<', '>']):
                    cond_left, operator, cond_right = split(cond)

                    for fact in known_facts:
                        fact_left, fact_operator, fact_right = split(fact)

                        if fact_left == cond_left:
                                value1 = float(fact_right)
                                value2 = float(cond_right)
                                if operator == '=' and fact_operator == '=' and value1 == value2:
                                    matched = True
                                elif operator == '<' and value1 < value2:
                                    matched = True
                                elif operator == '>' and value1 > value2:
                                    matched = True
                                break
                else:
                    if cond in known_facts:
                        matched = True

                if not matched:
                    break
            else:
                # add to facts after checking all conditions
                if rule['conclusion'] not in known_facts:
                    print(f"-> New Fact: {rule['conclusion']}")
                    known_facts.add(rule['conclusion'])
                    new_fact = True

                    if goal and rule['conclusion'] == goal:
                        print(f"\nGoal '{goal}' reached!")
                        return known_facts

        cycle += 1

    print(f"\nFinal Facts: {sorted(known_facts)}")
    if goal and goal not in known_facts:
        print(f"\nGoal '{goal}' not reached.")


    return known_facts



def split(condition):
    for op in ['=', '<', '>']:
        if op in condition:
            left, right = condition.split(op)
            return left.strip(), op, right.strip()
    return condition, None, None




goal = "citrus_fruit"
f =  forward(read_rules(), read_facts(), goal)


### BACKWARD CHAINING

In [ ]:
def read_facts(file):
    facts = []
    file = open(file,'r')
    for line in file:
        facts.append(line.strip().replace('skin_smell','perfumed'))
    processed_fact = {}
    for fact in facts:
        if '=' in fact:
            key, value = fact.split('=')
            processed_fact[key.strip()] = value.strip()
        elif 'is' in fact:
            key, value = fact.split('is')
            processed_fact[key.strip()] = value.strip()
        else:
            processed_fact[fact.strip()] = True
    # print(processed_fact)
    return processed_fact

def read_rules(file):
    rules=[]
    rule = {}
    file = open(file,'r')
    for line in file:
        if "IF" in line and "THEN" in line:
            _, item = line.split("IF")
            condition, conclusion = item.split("THEN")
            rule[condition.strip().replace('AND','and').replace('OR','or')] = conclusion.strip()
    # print(rule)
    return rule

def split_and(condition):
    split =[]
    if 'and' in condition:
        split = condition.split(' and ')
        split = [word.strip() for word in split]
    return split

def split_or(condition):
    split = []
    if 'or' in condition:
        split = condition.split(' or ')
        split = [word.strip() for word in split]
    return split

def add_to_KB(item):
    print(f"***Adding {item} to KB***")
    if 'is' in item:
        key, val = item.split('is')
        knowledge_base[key.strip()] = val.strip()
    else:
        knowledge_base[item] = True
    print(f"***Current KB: {knowledge_base}***")
    return knowledge_base

rules = read_rules("rules.txt")
knowledge_base = read_facts("facts.txt")
goal = "citrus_fruit"

def back_chaining(knowledge_base, goal, level) -> bool:
    indentM = "    " * level
    indentR = "    " * (level+1)
    print(f"{indentM}Matching {goal}")
    i = 1
    for key, val in rules.items():
        if val == goal:
            print(f"{indentR}R{i}: IF {key} THEN {val}")
            or_split = split_or(key)
            and_split = split_and(key)
            if len(or_split) > 1 and len(and_split) <= 1:
                found = False
                for word in or_split:
                    # print(word)
                    # print("YAYAYAYAYAY")
                    if word.split()[0] in knowledge_base:
                        knowledge_base = add_to_KB(goal)
                        found = True
                        return True
                if found == False:
                    for word in or_split:
                        if back_chaining(knowledge_base, word, level+1):

                            print(f"{indentR}Return to R{i}")


                            knowledge_base = add_to_KB(goal)
                            return True
            elif len(or_split) <= 1 and len(and_split) > 1:
                key_copy = key.replace('is', '==')
                for word in and_split:
                    if word.split()[0] in knowledge_base:
                        key_copy = key_copy.replace(word.split()[0],str(knowledge_base[word.split()[0]]))
                    elif word.split()[0] not in knowledge_base:
                        if back_chaining(knowledge_base, word, level+1):

                            print(f"{indentR}Return to R{i}")

                            knowledge_base = add_to_KB(word)
                            key_copy = key_copy.replace(word.split()[0],str(knowledge_base[word.split()[0]]))
                        else:
                            return False
                # print(key_copy)
                for item in key_copy.split():
                    if isinstance(item, str):
                        key_copy = key_copy.replace(item, "'item'")

                if eval(key_copy):
                    knowledge_base = add_to_KB(val)
                    return True
                else:
                    return False
        # else:
        #     continue
        i += 1

if back_chaining(knowledge_base, goal, 0):
    print("GOAL REACHED :D")
else:
    print("Goal couldn't be reached :(")